In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from PIL import Image
import torch
from transformers import VisionEncoderDecoderModel, TrOCRProcessor, Trainer, TrainingArguments, TrainerCallback
import evaluate
from tqdm import tqdm

In [2]:
test_size = 0.2

training_args = TrainingArguments(
    output_dir="./spanish_ocr_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    save_total_limit=2,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="cer",
    greater_is_better=False,
    report_to="none",
)

In [3]:
class SpanishOCRDataset(Dataset):
    def __init__(self, dataframe, images_dir, processor, max_target_length=128):
        self.data = dataframe.reset_index(drop=True)
        self.images_dir = images_dir
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.images_dir, row['image_path'])
        text = row['text']

        image = Image.open(img_path).convert("RGB")
        pixel_values = self.processor(images=image, return_tensors="pt").pixel_values.squeeze(0)
        labels = self.processor.tokenizer(
            text,
            padding="max_length",
            max_length=self.max_target_length,
            truncation=True,
            return_tensors="pt"
        ).input_ids.squeeze(0)
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        return {
            "pixel_values": pixel_values,
            "labels": labels,
        }
    
class TQDMProgressBar(TrainerCallback):
    def __init__(self):
        self.progress_bar = None

    def on_train_begin(self, args, state, control, **kwargs):
        self.progress_bar = tqdm(total=state.max_steps, desc="Training Progress", dynamic_ncols=True)

    def on_log(self, args, state, control, logs=None, **kwargs):
        if self.progress_bar is not None and logs is not None:
            loss = logs.get('loss', None)
            if loss is not None:
                desc = f"Epoch {int(state.epoch)} | Step {state.global_step} | Loss {loss:.4f}"
            else:
                desc = f"Epoch {int(state.epoch)} | Step {state.global_step}"

            if 'eval_cer' in logs:
                desc += f" | CER {logs['eval_cer']:.4f}"
            if 'eval_wer' in logs:
                desc += f" | WER {logs['eval_wer']:.4f}"

            self.progress_bar.set_description(desc)

    def on_step_end(self, args, state, control, **kwargs):
        if self.progress_bar is not None:
            self.progress_bar.update(1)

    def on_train_end(self, args, state, control, **kwargs):
        if self.progress_bar is not None:
            self.progress_bar.close()

In [4]:
def build_model():
    processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-printed")
    model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-printed")

    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")
        
    model = model.to(device)
    model.config.decoder_start_token_id = processor.tokenizer.cls_token_id
    model.config.pad_token_id = processor.tokenizer.pad_token_id
    print("Model built")
    return processor, model

def build_datasets(processor, test_size=0.1):
    df = pd.read_csv("data/spanish_data.csv")
    train_df, val_df = train_test_split(df, test_size=test_size, random_state=42)

    train_dataset = SpanishOCRDataset(train_df, "data/images/", processor)
    val_dataset = SpanishOCRDataset(val_df, "data/images/", processor)

    print("Dataset built")

    return train_dataset, val_dataset

def build_compute_metrics(processor):
    def compute_metrics(pred):
        pred_ids = pred.predictions
        label_ids = pred.label_ids

        if isinstance(pred_ids, tuple):
            pred_ids = pred_ids[0]
        pred_ids = torch.tensor(pred_ids)
        pred_ids = torch.argmax(pred_ids, dim=-1)

        label_ids = torch.tensor(label_ids)
        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

        pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

        cer_metric = evaluate.load("cer")
        wer_metric = evaluate.load("wer")

        cer = cer_metric.compute(predictions=pred_str, references=label_str)
        wer = wer_metric.compute(predictions=pred_str, references=label_str)

        return {"cer": cer, "wer": wer}

    return compute_metrics

In [5]:
processor, model = build_model()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}



Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "eos_token_id": 2,
  "init_std": 0.02,
  "is_decoder": true,
  "layernorm_embedding": true,
  "max_position_embeddings": 512,
  "model_type": "trocr",
  "pad_token_id": 1,
  "scale_embedding": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3",
  "use_cache": false,
  "use_learned_position_embeddings": true,
  "vocab_size": 50265
}



Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-printed and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model built


In [6]:
train_dataset, val_dataset = build_datasets(processor)

Dataset built


In [7]:
compute_metrics = build_compute_metrics(processor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[TQDMProgressBar()],
)
trainer.train()

Training Progress:   0%|          | 0/1620 [00:00<?, ?it/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Training Progress:   0%|          | 1/1620 [00:23<10:34:24, 23.51s/it]

Epoch,Training Loss,Validation Loss,Cer,Wer
1,No log,6.115152,0.856703,1.002494
2,No log,5.531605,1.052747,1.032419
3,No log,5.473433,0.936703,1.009975
4,No log,5.163579,0.927912,1.062344
5,No log,5.302195,0.615824,0.917706
6,No log,4.537141,0.792088,1.216958
7,No log,4.446496,0.710330,1.206983
8,No log,4.422568,0.875165,1.294264
9,No log,4.295557,0.631209,1.132170
10,3.364100,4.086585,0.923956,1.431421


Training Progress:   0%|          | 2/1620 [00:24<4:35:16, 10.21s/it] 

...

Epoch 30 | Step 1620 | CER 0.5952 | WER 1.2195: 100%|██████████| 1620/1620 [29:33<00:00,  1.46it/s]

There were missing keys in the checkpoint model loaded: ['decoder.output_projection.weight'].


Epoch 30 | Step 1620: 100%|██████████| 1620/1620 [29:40<00:00,  1.46it/s]                          

Epoch 30 | Step 1620: 100%|██████████| 1620/1620 [29:40<00:00,  1.10s/it]

TrainOutput(global_step=1620, training_loss=1.2353943509820067, metrics={'train_runtime': 1780.9628, 'train_samples_per_second': 7.21, 'train_steps_per_second': 0.91, 'total_flos': 9.60798185238823e+18, 'train_loss': 1.2353943509820067, 'epoch': 30.0})

In [8]:
model.save_pretrained("./spanish_ocr_model")
processor.save_pretrained("./spanish_ocr_model")

[]